In [1]:
!git clone https://github.com/jalpv1//wiki-art-selected-resized-downsampled
!rm -rf .//wiki-art-selected-resized-downsampled/.git
model_name ="densenet_tr1"
cv = 5

fatal: destination path 'wiki-art-selected-resized-downsampled' already exists and is not an empty directory.


In [ ]:
!pip install torchsampler
!pip install xgboost
!pip install torchmetrics
!pip install pytorch_lightning
!pip install optuna
!pip install memory_profiler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.3/231.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from torch.utils.data import WeightedRandomSampler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
import time
import os
import copy
from torch.utils.data import DataLoader, TensorDataset, WeightedRandomSampler
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
import cv2
import numpy as np
import imageio as io
from PIL import Image
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

import torchmetrics
import pytorch_lightning as pl

import data_loader as dl
import experiments as e


!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
class_names = ['Abstract_Expressionism','Baroque','Cubism', 'Fauvism', 'Impressionism','Minimalism','Naive_Art_Primitivism','Pointillism','Rococo','Ukiyo_e']
device = 'cuda'

num_classes = 10


Python 3.11.6
/usr/bin/sh: line 1: nvcc: command not found
Source files will be saved in "/tmp/tmppxiqw1oh".


In [7]:
class ArtStyleDataset(Dataset):
  def __init__(self, root_dir, transform = None, transform_tensor = None):
     self.imgs = ImageFolder(root_dir)
     self.root_dir = root_dir
     self.transform = transform
     self.transform_tensor = transform_tensor


  def __len__(self):
        return len(self.imgs)

  def __getitem__(self, idx):
        sample, label = self.imgs[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, label
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
data = ArtStyleDataset('wiki-art-selected-resized-downsampled', transform)


In [4]:
class Model (torch.nn.Module):
  # def __init__(self, layer_name: str = 'features.4'): layer1
  def __init__(self, layer_name: str = 'features.transition1'):
     super(Model, self).__init__()
     self.model = models.densenet121(pretrained=True)
     self.layer_name = layer_name

     self.activations = dict()

     def hook(module, input, output):
        self.activations[layer_name] = output

     self.model.get_submodule(layer_name).register_forward_hook(hook)

  def forward(self, x):
        # return self.model(x)
        # pass
        _ = self.model(x)

        return self.activations[self.layer_name]

In [5]:
model = Model().to(device)
dataset = dl.data_load(data)
train_data , train_labels,map_img_feature = dl.extract_features(model, dataset)
dl.save_pickle(train_data,train_labels, model_name)



/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /home//.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 102MB/s] 


(128,)
(227,)
----------------------------------
(128,)


/opt/conda/lib/python3.11/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
----------------------------------
(128,)
(227,)
-------------

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

#dl.save_pickle(train_data,train_labels, model_name)


X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

train_labels = np.array(train_labels)
train_labels = train_labels.ravel()


print(f"Train: {len(X_train)}, Test: {len(X_test)}")


Train: 14804, Test: 3701


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle

with open('traindata_resnet18_2.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('train_labels_resnet18_2.pickle', 'rb') as handle:
    train_labels = pickle.load(handle)

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

train_labels = np.array(train_labels)
train_labels = train_labels.ravel()


print(f"Train: {len(X_train)}, Test: {len(X_test)}")

Train: 14804, Test: 3701


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import  ConfusionMatrixDisplay
from xgboost import XGBRFClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    classification_report, roc_curve, auc, precision_recall_curve, confusion_matrix
)
import xgboost


models = {
    "Random Forest": RandomForestClassifier(n_estimators = 500, random_state=42),
    "SVM": SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42),
    "KNN":KNeighborsClassifier(n_neighbors=30, metric='cosine'),
    "MLP": MLPClassifier(
    hidden_layer_sizes=(300, 150, 75),
                    max_iter=500,
                    solver='adam',
                    learning_rate="adaptive",
                    learning_rate_init=0.001,
                    early_stopping=True,
                    random_state=42),
    "XgBoost":  XGBRFClassifier(n_estimators=500, max_depth=20, random_state=42)
}

for name in models.keys():
  print(f"Model: {name}")
  avg = e.kfold_crossvalidation(train_data, train_labels, 5,  models[name], 1)
  print(avg)





Model: Random Forest
       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.828601  0.827074  0.825833  0.827074   98.574785   
1         2   0.825818  0.822751  0.821793  0.822751   95.888840   
2         3   0.841486  0.839233  0.838427  0.839233   96.159021   
3         4   0.830189  0.825993  0.825557  0.825993   96.622875   
4         5   0.836821  0.834639  0.833581  0.834639   96.633859   
5   Average   0.832583  0.829938  0.829038  0.829938   96.775876   
6  1.581139   0.006414  0.006781  0.006776  0.006781    1.054395   

   Prediction time  
0         0.312248  
1         0.335319  
2         0.356712  
3         0.365270  
4         0.352006  
5         0.344311  
6         0.020983  
Model: SVM
       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.693866  0.691435  0.689732  0.691435   11.858569   
1         2   0.696360  0.696028  0.694021  0.696028   11.756085   
2         3   0.706276  0.705485  0.703522  0.70548

In [9]:
model_name ="densenet_tr2"

class Model (torch.nn.Module):
  # def __init__(self, layer_name: str = 'features.4'): layer1
  def __init__(self, layer_name: str = 'features.transition2'):
     super(Model, self).__init__()
     self.model = models.densenet121(pretrained=True)
     self.layer_name = layer_name

     self.activations = dict()

     def hook(module, input, output):
        self.activations[layer_name] = output

     self.model.get_submodule(layer_name).register_forward_hook(hook)

  def forward(self, x):
        # return self.model(x)
        # pass
        _ = self.model(x)

        return self.activations[self.layer_name]

In [ ]:
model = Model().to(device)
dataset = dl.data_load(data)
train_data , train_labels,map_img_feature = dl.extract_features(model, dataset)
dl.save_pickle(train_data,train_labels, model_name)


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle


with open('X_densenet_tr2.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('Y_densenet_tr2.pickle', 'rb') as handle:
    train_labels = pickle.load(handle)

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

train_labels = np.array(train_labels)
train_labels = train_labels.ravel()


print(f"Train: {len(X_train)}, Test: {len(X_test)}")

Train: 14804, Test: 3701


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import  ConfusionMatrixDisplay
from xgboost import XGBRFClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    classification_report, roc_curve, auc, precision_recall_curve, confusion_matrix
)
import xgboost


models = {
    "Random Forest": RandomForestClassifier(n_estimators = 500, random_state=42),
    "SVM": SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42),
    "KNN":KNeighborsClassifier(n_neighbors=30, metric='cosine'),
    "MLP": MLPClassifier(
    hidden_layer_sizes=(300, 150, 75),
                    max_iter=500,
                    solver='adam',
                    learning_rate="adaptive",
                    learning_rate_init=0.001,
                    early_stopping=True,
                    random_state=42),
    "XgBoost":  XGBRFClassifier(n_estimators=500, max_depth=20, random_state=42)
}

for name in models.keys():
  print(f"Model: {name}")
  avg = e.kfold_crossvalidation(train_data, train_labels, 5,  models[name], 1)
  print(avg)





Model: Random Forest


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usi

       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.824488  0.819508  0.818512  0.819508  122.177676   
1         2   0.831198  0.828155  0.827612  0.828155  119.169928   
2         3   0.828350  0.823291  0.822846  0.823291  119.083178   
3         4   0.838556  0.834369  0.833828  0.834369  119.248176   
4         5   0.833920  0.831397  0.830760  0.831397  118.639514   
5   Average   0.831302  0.827344  0.826711  0.827344  119.663694   
6  1.581139   0.005353  0.006005  0.006124  0.006005    1.425001   

   Prediction time  
0         0.572644  
1         0.433272  
2         0.387089  
3         0.379004  
4         0.396726  
5         0.433747  
6         0.080369  
Model: SVM


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.769913  0.766820  0.765276  0.766820   18.448667   
1         2   0.773758  0.772764  0.771617  0.772764   17.919780   
2         3   0.764458  0.760605  0.759500  0.760605   18.681987   
3         4   0.770465  0.767630  0.766623  0.767630   18.269077   
4         5   0.770709  0.768711  0.766920  0.768711   17.810263   
5   Average   0.769861  0.767306  0.765987  0.767306   18.225955   
6  1.581139   0.003371  0.004388  0.004346  0.004388    0.362618   

   Prediction time  
0        11.359083  
1        11.678809  
2        12.063031  
3        11.096432  
4        11.181655  
5        11.475802  
6         0.396900  
Model: KNN


/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for 

       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.599343  0.573629  0.566785  0.573629    0.010075   
1         2   0.609196  0.576331  0.568687  0.576331    0.008560   
2         3   0.589103  0.563631  0.553847  0.563631    0.008364   
3         4   0.611988  0.585247  0.577012  0.585247    0.008119   
4         5   0.597583  0.576061  0.567695  0.576061    0.008609   
5   Average   0.601443  0.574980  0.566805  0.574980    0.008745   
6  1.581139   0.009259  0.007730  0.008313  0.007730    0.000768   

   Prediction time  
0         1.000196  
1         1.004817  
2         1.039331  
3         1.016577  
4         1.000160  
5         1.012216  
6         0.016574  
Model: MLP


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed w

       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.807446  0.807620  0.806907  0.807620   15.865801   
1         2   0.806511  0.805728  0.805381  0.805728   11.102519   
2         3   0.797858  0.797082  0.797020  0.797082   12.009629   
3         4   0.804652  0.805188  0.804656  0.805188   12.176341   
4         5   0.816119  0.815996  0.815566  0.815996   20.486393   
5   Average   0.806517  0.806323  0.805906  0.806323   14.328137   
6  1.581139   0.006548  0.006751  0.006617  0.006751    3.895473   

   Prediction time  
0         0.017062  
1         0.012053  
2         0.026880  
3         0.017640  
4         0.012436  
5         0.017214  
6         0.005981  
Model: XgBoost


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import  ConfusionMatrixDisplay
from xgboost import XGBRFClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    classification_report, roc_curve, auc, precision_recall_curve, confusion_matrix
)
import xgboost


models = {
    "XgBoost":  XGBRFClassifier(n_estimators=500, max_depth=20, random_state=42)
}

for name in models.keys():
  print(f"Model: {name}")
  avg = e.kfold_crossvalidation(train_data, train_labels, 5,  models[name], 1)
  print(avg)

Model: XgBoost
       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.813669  0.810862  0.810116  0.810862  322.995770   
1         2   0.824093  0.823561  0.823285  0.823561  365.674522   
2         3   0.819840  0.818157  0.817443  0.818157  336.094000   
3         4   0.832292  0.831127  0.830274  0.831127  383.785963   
4         5   0.827503  0.826533  0.825973  0.826533  363.341375   
5   Average   0.823479  0.822048  0.821418  0.822048  354.378326   
6  1.581139   0.007137  0.007828  0.007846  0.007828   24.453685   

   Prediction time  
0         0.261394  
1         0.217274  
2         0.247468  
3         0.373685  
4         0.179190  
5         0.255802  
6         0.073060  


In [8]:
model_name ="densenet_tr3"

class Model (torch.nn.Module):
  # def __init__(self, layer_name: str = 'features.4'): layer1
  def __init__(self, layer_name: str = 'features.transition3'):
     super(Model, self).__init__()
     self.model = models.densenet121(pretrained=True)
     self.layer_name = layer_name

     self.activations = dict()

     def hook(module, input, output):
        self.activations[layer_name] = output

     self.model.get_submodule(layer_name).register_forward_hook(hook)

  def forward(self, x):
        # return self.model(x)
        # pass
        _ = self.model(x)

        return self.activations[self.layer_name]

model = Model().to(device)
dataset = dl.data_load(data)
train_data , train_labels,map_img_feature = dl.extract_features(model, dataset)
dl.save_pickle(train_data,train_labels, model_name)


(512,)
(611,)
----------------------------------
(512,)


/opt/conda/lib/python3.11/site-packages/skimage/feature/texture.py:385: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
----------------------------------
(512,)
(611,)
-------------

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import  ConfusionMatrixDisplay
from xgboost import XGBRFClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    classification_report, roc_curve, auc, precision_recall_curve, confusion_matrix
)
import xgboost


models = {
    "Random Forest": RandomForestClassifier(n_estimators = 500, random_state=42),
    "SVM": SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42),
    "KNN":KNeighborsClassifier(n_neighbors=30, metric='cosine'),
    "MLP": MLPClassifier(
    hidden_layer_sizes=(300, 150, 75),
                    max_iter=500,
                    solver='adam',
                    learning_rate="adaptive",
                    learning_rate_init=0.001,
                    early_stopping=True,
                    random_state=42),
    "XgBoost":  XGBRFClassifier(n_estimators=500, max_depth=20, random_state=42)
}

for name in models.keys():
  print(f"Model: {name}")
  avg = e.kfold_crossvalidation(train_data, train_labels, 5,  models[name], 1)
  print(avg)





Model: Random Forest


/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/conda/lib/python3.11/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usi

       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.824021  0.817617  0.817361  0.817617  165.417287   
1         2   0.825752  0.819778  0.818607  0.819778  163.780493   
2         3   0.829787  0.821940  0.821244  0.821940  165.426044   
3         4   0.811336  0.804647  0.804206  0.804647  163.720107   
4         5   0.824144  0.816806  0.816117  0.816806  163.811771   
5   Average   0.823008  0.816158  0.815507  0.816158  164.431140   
6  1.581139   0.006929  0.006738  0.006596  0.006738    0.904827   

   Prediction time  
0         0.509815  
1         0.526714  
2         0.476531  
3         0.482188  
4         0.471342  
5         0.493318  
6         0.023854  
Model: SVM


/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for exam

       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.790742  0.786814  0.785791  0.786814   38.007847   
1         2   0.787544  0.783842  0.782054  0.783842   37.473266   
2         3   0.793821  0.787355  0.786223  0.787355   37.507749   
3         4   0.773872  0.770062  0.769003  0.770062   36.452308   
4         5   0.801251  0.796271  0.795465  0.796271   37.623404   
5   Average   0.789446  0.784869  0.783707  0.784869   37.412915   
6  1.581139   0.010079  0.009490  0.009590  0.009490    0.577419   

   Prediction time  
0        19.919086  
1        19.048409  
2        19.502656  
3        18.312058  
4        18.930710  
5        19.142584  
6         0.607482  
Model: KNN


/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/opt/conda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:239: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for 

       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.502626  0.489597  0.480873  0.489597    0.052121   
1         2   0.501908  0.489057  0.481499  0.489057    0.056392   
2         3   0.509395  0.498514  0.489578  0.498514    0.054375   
3         4   0.497375  0.483923  0.472945  0.483923    0.084865   
4         5   0.520684  0.500135  0.491992  0.500135    0.051349   
5   Average   0.506397  0.492245  0.483377  0.492245    0.059820   
6  1.581139   0.009067  0.006855  0.007604  0.006855    0.014139   

   Prediction time  
0         1.240437  
1         1.190613  
2         1.195535  
3         1.277502  
4         1.200678  
5         1.220953  
6         0.037266  
Model: MLP


/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1124: DataConversionWarning: A column-vector y was passed w

       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.801230  0.800594  0.800481  0.800594   13.566556   
1         2   0.802998  0.801675  0.801025  0.801675   17.642817   
2         3   0.800541  0.800865  0.800408  0.800865   15.872226   
3         4   0.795685  0.796001  0.795373  0.796001   16.751440   
4         5   0.819580  0.818968  0.818854  0.818968   17.914780   
5   Average   0.804007  0.803621  0.803228  0.803621   16.349564   
6  1.581139   0.009117  0.008862  0.009031  0.008862    1.750492   

   Prediction time  
0         0.041930  
1         0.019947  
2         0.017597  
3         0.013720  
4         0.028789  
5         0.024397  
6         0.011256  
Model: XgBoost


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle


with open('X_densenet_tr3.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('Y_densenet_tr3.pickle', 'rb') as handle:
    train_labels = pickle.load(handle)

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
train_data_scaled = scaler.transform(train_data)

train_labels = np.array(train_labels)
train_labels = train_labels.ravel()


print(f"Train: {len(X_train)}, Test: {len(X_test)}")

Train: 14804, Test: 3701


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import  ConfusionMatrixDisplay
from xgboost import XGBRFClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    classification_report, roc_curve, auc, precision_recall_curve, confusion_matrix
)
import xgboost


models = {
    "XgBoost":  XGBRFClassifier(n_estimators=500, max_depth=20, random_state=42)
}

for name in models.keys():
  print(f"Model: {name}")
  avg = e.kfold_crossvalidation(train_data, train_labels, 5,  models[name], 1)
  print(avg)





Model: XgBoost
       Fold  Precision    Recall        F1  Accuracy  Train time  \
0         1   0.812996  0.811132  0.810438  0.811132  680.743345   
1         2   0.810602  0.808971  0.807484  0.808971  617.596382   
2         3   0.821246  0.818427  0.817642  0.818427  783.456683   
3         4   0.801361  0.800054  0.799408  0.800054  588.073098   
4         5   0.820600  0.817617  0.816889  0.817617  753.081342   
5   Average   0.813361  0.811240  0.810372  0.811240  684.590170   
6  1.581139   0.008160  0.007460  0.007481  0.007460   84.089445   

   Prediction time  
0         0.259577  
1         0.249410  
2         0.283110  
3         0.289362  
4         0.296825  
5         0.275657  
6         0.020242  


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import optuna 
import logging

with open('X_densenet_tr3.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('Y_densenet_tr3.pickle', 'rb') as handle:
    train_labels = pickle.load(handle)

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
train_data_scaled = scaler.transform(train_data)

train_labels = np.array(train_labels)
train_labels = train_labels.ravel()


print(f"Train: {len(X_train)}, Test: {len(X_test)}")

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train: 14804, Test: 3701


In [4]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.datasets import make_classification
from sklearn.svm import SVC

optuna.logging.enable_propagation()
optuna.logging.set_verbosity(optuna.logging.INFO)

def objective(trial):
    kernel = trial.suggest_categorical("kernel", ["linear", "rbf", "poly"])
    C = trial.suggest_float("C", 0.01, 100)

    if kernel == "rbf" or kernel == "poly":
        gamma = trial.suggest_float("gamma", 0.0001, 10)
    else:
        gamma = "scale"  
    
    if kernel == "poly":
        degree = trial.suggest_int("degree", 2, 5)
    else:
        degree = 3  

    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree, random_state=42)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    
    scorer = make_scorer(f1_score, average="weighted")
    scores = cross_val_score(model, train_data_scaled, train_labels, scoring=scorer, cv=skf, n_jobs=4)
    score = np.mean(scores)
    logging.info(f"Trial {trial.number} finished with value: {score:.5f} and parameters: {trial.params}")

    return score


study = optuna.create_study( study_name= "svc_dn_3_2",storage="sqlite:///svc_dn_3_2.db",direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=50)

df = study.trials_dataframe()
df.to_csv("orsvc_dn_3.csv", index=False)
logging.info("Optuna study completed. Results saved to optuna_results.csv.")

best_params = study.best_params
logging.info(f"Best parameters: {best_params}")
print("Best parameters for RF:", study.best_params)
print("Best F1-score for RF:", study.best_value)

plt.figure(figsize=(8, 6))
optuna.visualization.matplotlib.plot_optimization_history(study)
plt.title("Optuna Optimization History")
plt.savefig("ohsvc_dn_3.png")
plt.show()

plt.figure(figsize=(8, 6))
optuna.visualization.matplotlib.plot_param_importances(study)
#plt.title("Hyperparameter Importance")
plt.savefig("pisvc_dn_3.png")
plt.show()

logging.info("Plots saved: optimization_history.png and param_importance.png")


[I 2025-03-29 22:35:46,774] A new study created in RDB with name: svc_dn_3_2
[I 2025-03-30 02:18:27,535] Trial 0 finished with value: 0.7556089786890278 and parameters: {'kernel': 'linear', 'C': 51.27072548662446}. Best is trial 0 with value: 0.7556089786890278.
INFO:optuna.study.study:Trial 0 finished with value: 0.7556089786890278 and parameters: {'kernel': 'linear', 'C': 51.27072548662446}. Best is trial 0 with value: 0.7556089786890278.
[I 2025-03-30 02:20:54,369] Trial 1 finished with value: 0.8195577988648605 and parameters: {'kernel': 'poly', 'C': 57.33855678988371, 'gamma': 5.083760134889122, 'degree': 3}. Best is trial 1 with value: 0.8195577988648605.
INFO:optuna.study.study:Trial 1 finished with value: 0.8195577988648605 and parameters: {'kernel': 'poly', 'C': 57.33855678988371, 'gamma': 5.083760134889122, 'degree': 3}. Best is trial 1 with value: 0.8195577988648605.
[I 2025-03-30 02:24:28,272] Trial 2 finished with value: 0.04014416492090159 and parameters: {'kernel': 'rbf'

KeyboardInterrupt: 

In [7]:
import optuna

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import logging here

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
import optuna 
import logging

with open('X_densenet_tr2.pickle', 'rb') as handle:
    train_data = pickle.load(handle)
with open('Y_densenet_tr2.pickle', 'rb') as handle:
    train_labels = pickle.load(handle)

X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
train_data_scaled = scaler.transform(train_data)

train_labels = np.array(train_labels)
train_labels = train_labels.ravel()


print(f"Train: {len(X_train)}, Test: {len(X_test)}")

Train: 14804, Test: 3701


In [6]:
import optuna
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from joblib import parallel_backend
import numpy as np
import logging
import xgboost as xgb


def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1200, step=100),  
        "max_depth": trial.suggest_int("max_depth", 3, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 10, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-2, 10, log=True),
        "objective": "multi:softmax",  
        "eval_metric": "mlogloss",
        "tree_method": "hist",  
    }

    model = xgb.XGBClassifier(**params, random_state=42, n_jobs=-1)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    
    scorer = make_scorer(f1_score, average="weighted")
    scores = cross_val_score(model, train_data_scaled, train_labels, scoring=scorer, cv=skf, n_jobs=4)
    score = np.mean(scores)
    logging.info(f"Trial {trial.number} finished with value: {score:.5f} and parameters: {trial.params}")

    return score


study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner(), study_name= "xg_dn_2",storage="sqlite:///xg_dn_2.db")
study.optimize(objective, n_trials=50, n_jobs=16)

df = study.trials_dataframe()
df.to_csv("or_xg_dn_2.csv", index=False)
logging.info("Optuna study completed. Results saved to optuna_results.csv.")

best_params = study.best_params
logging.info(f"Best parameters: {best_params}")
print("Best parameters for classifier:", study.best_params)
print("Best F1-score for classifier:", study.best_value)

plt.figure(figsize=(8, 6))
optuna.visualization.matplotlib.plot_optimization_history(study)
plt.title("Optuna Optimization History")
plt.savefig("op_xg_dn_2.png")
plt.show()

plt.figure(figsize=(8, 6))
optuna.visualization.matplotlib.plot_param_importances(study)
plt.savefig("pi_xg_dn_2.png")
plt.show()

logging.info("Plots saved: optimization_history.png and param_importance.png")


[I 2025-03-30 06:18:12,109] A new study created in RDB with name: xg_dn_2
INFO:optuna.storages._rdb.storage:A new study created in RDB with name: xg_dn_2
[I 2025-03-30 06:36:09,063] Trial 5 finished with value: 0.834014446295412 and parameters: {'n_estimators': 600, 'max_depth': 17, 'learning_rate': 0.061099699279845494, 'subsample': 0.5282937352197772, 'colsample_bytree': 0.8686851879903447, 'gamma': 1.6842333972715384, 'reg_lambda': 6.919089840787873, 'reg_alpha': 0.021615572156485307}. Best is trial 5 with value: 0.834014446295412.
INFO:optuna.study.study:Trial 5 finished with value: 0.834014446295412 and parameters: {'n_estimators': 600, 'max_depth': 17, 'learning_rate': 0.061099699279845494, 'subsample': 0.5282937352197772, 'colsample_bytree': 0.8686851879903447, 'gamma': 1.6842333972715384, 'reg_lambda': 6.919089840787873, 'reg_alpha': 0.021615572156485307}. Best is trial 5 with value: 0.834014446295412.
[I 2025-03-30 06:44:41,468] Trial 6 finished with value: 0.6494510784019181 

KeyboardInterrupt: 

In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from joblib import parallel_backend
import numpy as np
import logging
import xgboost as xgb


def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1200, step=100),  
        "max_depth": trial.suggest_int("max_depth", 3, 50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 10, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-2, 10, log=True),
        "objective": "multi:softmax",  
        "eval_metric": "mlogloss",
        "tree_method": "hist",  
    }

    model = xgb.XGBClassifier(**params, random_state=42, n_jobs=-1)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    
    scorer = make_scorer(f1_score, average="weighted")
    scores = cross_val_score(model, train_data_scaled, train_labels, scoring=scorer, cv=skf, n_jobs=4)
    score = np.mean(scores)
    logging.info(f"Trial {trial.number} finished with value: {score:.5f} and parameters: {trial.params}")

    return score


study = optuna.load_study(study_name= "xg_dn_2",storage="sqlite:///xg_dn_2.db")
study.optimize(objective, n_trials=44, n_jobs=16)

df = study.trials_dataframe()
df.to_csv("or_xg_dn_2.csv", index=False)
logging.info("Optuna study completed. Results saved to optuna_results.csv.")

best_params = study.best_params
logging.info(f"Best parameters: {best_params}")
print("Best parameters for classifier:", study.best_params)
print("Best F1-score for classifier:", study.best_value)

plt.figure(figsize=(8, 6))
optuna.visualization.matplotlib.plot_optimization_history(study)
plt.title("Optuna Optimization History")
plt.savefig("op_xg_dn_2.png")
plt.show()

plt.figure(figsize=(8, 6))
optuna.visualization.matplotlib.plot_param_importances(study)
plt.savefig("pi_xg_dn_2.png")
plt.show()

logging.info("Plots saved: optimization_history.png and param_importance.png")


[I 2025-03-30 07:00:31,970] Trial 23 finished with value: 0.8375479487820293 and parameters: {'n_estimators': 400, 'max_depth': 8, 'learning_rate': 0.09587102517812422, 'subsample': 0.9093043932749565, 'colsample_bytree': 0.7307990039250936, 'gamma': 1.4493847286506845, 'reg_lambda': 0.016665173326806438, 'reg_alpha': 0.33976559684991603}. Best is trial 23 with value: 0.8375479487820293.
INFO:optuna.study.study:Trial 23 finished with value: 0.8375479487820293 and parameters: {'n_estimators': 400, 'max_depth': 8, 'learning_rate': 0.09587102517812422, 'subsample': 0.9093043932749565, 'colsample_bytree': 0.7307990039250936, 'gamma': 1.4493847286506845, 'reg_lambda': 0.016665173326806438, 'reg_alpha': 0.33976559684991603}. Best is trial 23 with value: 0.8375479487820293.
[I 2025-03-30 07:27:41,068] Trial 22 finished with value: 0.7664743078358073 and parameters: {'n_estimators': 600, 'max_depth': 43, 'learning_rate': 0.010431425205884158, 'subsample': 0.5316758031113642, 'colsample_bytree'

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from joblib import parallel_backend
import numpy as np
import logging


def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1200, step=100),  # More trees for better performance
        "max_depth": trial.suggest_int("max_depth", 3, 50),  # Shallow trees avoid overfitting, deeper trees capture complexity
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),  # Regularization to prevent overfitting
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),  # Smaller leaf size = more flexible tree
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),  # Feature selection
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),  # Whether to use bootstrapping
        "criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),  # Splitting criteria
    }

    model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    
    scorer = make_scorer(f1_score, average="weighted")
    scores = cross_val_score(model, train_data_scaled, train_labels, scoring=scorer, cv=skf, n_jobs=5)
    score = np.mean(scores)
    logging.info(f"Trial {trial.number} finished with value: {score:.5f} and parameters: {trial.params}")

    return score


study1 = optuna.create_study( study_name= "rf_dn_2",storage="sqlite:///rf_dn_2.db",direction="maximize", pruner=optuna.pruners.MedianPruner())
study1.optimize(objective, n_trials=50,n_jobs=5)

df = study1.trials_dataframe()
df.to_csv("rf_dn_2.csv", index=False)
logging.info("Optuna study completed. Results saved to optuna_results.csv.")

best_params = study1.best_params
logging.info(f"Best parameters: {best_params}")
print("Best parameters for classifier:", study1.best_params)
print("Best F1-score for classifier:", study1.best_value)

plt.figure(figsize=(8, 6))
optuna.visualization.matplotlib.plot_optimization_history(study1)
plt.title("Optuna Optimization History")
plt.savefig("op_rf_dn_2_5.png")
plt.show()

plt.figure(figsize=(8, 6))
optuna.visualization.matplotlib.plot_param_importances(study1)
#plt.title("Hyperparameter Importance")
plt.savefig("pi_rf_dn_2.png")
plt.show()

logging.info("Plots saved: optimization_history.png and param_importance.png")
